# collect all existing images into one place
- there are four root directory so i use os.walk()

In [1]:
import os
import pandas as pd

dir_path = "./ORAND-CAR-2014/"

CAR_A_test_images_path = dir_path + 'CAR-A/a_test_images/'
CAR_A_train_images_path = dir_path + 'CAR-A/a_train_images/'
CAR_B_test_images_path = dir_path + 'CAR-B/b_test_images/'
CAR_B_train_images_path = dir_path + 'CAR-B/b_train_images/'

image_paths = []
for root, dirs, files in os.walk(dir_path):
    for file in files:
        if file.endswith('.png'):
            image_path = os.path.join(root, file)
            image_paths.append(image_path)

print('Number of images:', len(image_paths))
print(image_paths[0])

Number of images: 11719
./ORAND-CAR-2014/CAR-A\a_test_images\a_car_007000.png


# From guide text file, I extract the dataframe of images_path and related label

In [19]:
CAR_A_test_text = dir_path + 'CAR-A/a_test_gt.txt'
CAR_A_train_text = dir_path + 'CAR-A/a_train_gt.txt'
CAR_B_test_text = dir_path + 'CAR-B/b_test_gt.txt'
CAR_B_train_text = dir_path + 'CAR-B/b_train_gt.txt'


def get_labels(image_dir,text_path):
  with open(text_path,'r') as f :
    lines = f.readlines()
  listt = []
  for line in lines :
    parts = line.strip().split("\t")
    listt.append([image_dir + parts[0],parts[1]])
  DF = pd.DataFrame(listt)
  DF = DF.rename(columns={0: 'image_path', 1: 'label'})
  return DF

a_test_label_df = get_labels(CAR_A_test_images_path,CAR_A_test_text)
a_train_label_df = get_labels(CAR_A_train_images_path,CAR_A_train_text)
b_test_label_df = get_labels(CAR_B_test_images_path,CAR_B_test_text)
b_train_label_df = get_labels(CAR_B_train_images_path,CAR_B_train_text)
all_labels = pd.concat([a_test_label_df , a_train_label_df , b_test_label_df , b_train_label_df],ignore_index=True)#.reset_index()
print(len(all_labels))
print(len(all_labels['image_path'].unique())) # no dupicate image name
all_labels.head()

11719
11719


,image_path,label
0,./ORAND-CAR-2014/CAR-A/a_test_images/a_car_007...,1500
1,./ORAND-CAR-2014/CAR-A/a_test_images/a_car_007...,5743
2,./ORAND-CAR-2014/CAR-A/a_test_images/a_car_007...,1056
3,./ORAND-CAR-2014/CAR-A/a_test_images/a_car_007...,1000
4,./ORAND-CAR-2014/CAR-A/a_test_images/a_car_007...,1000


> first checking some random images to know details and get familiar with data

> second let's go for resizing images because we cant store them in data frame
- for resizing i prefer to know mean of height and weight of images
- also we can try alternative sizes based on digits count (i mean size of all images is same but 4_digit number have twice full (meaningful)weight of 2_digit number
- maybe this help our model to predict better based on how many digit exist and what are these (we just try that i dont know realy)

> third is save them into dataframe and preprocess that to get ready for training


In [20]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

def plot_random_image(image_label_df):
  images = image_label_df['image_path']
  i = random.randint(0, len(images))
  target_image = images[i]
  print("label :",image_label_df['label'][i])
  print("target:",target_image)
  image_obj = Image.open(target_image)
  image = mpimg.imread(target_image)
  plt.imshow(image)
  plt.show()
  print("height : ",image_obj.height)
  print("width : ",image_obj.width)
  print("details : ",image_obj.getdata)


# Visualize Some Data

In [7]:
# from IPython.display import clear_output
# import time

# for i in range(5):
#     plot_random_image(all_labels)
#     time.sleep(2.5)
#     clear_output(wait=True)

# Now, Let's decide about resizing (it's what i don't know anything about that🤕)
- mean of height and width
- resize to large numbers for easy to work
- resize to common size (28*28 or something same)
- def a function for resizing in different sizes and test them
- also we can use size of images as a feature (because we know there are information in them)



# get details of sizes manually

In [8]:
# Get mean of height and width

# heights = 0
# widths = 0
# max_height = 0
# max_width = 0
# min_height = 800
# min_width = 800
# for image_path in all_labels['image_path']:
#   height = Image.open(image_path).height
#   width = Image.open(image_path).width
#   heights += height
#   widths += width
#   if height > max_height: max_height = height
#   if width > max_width: max_width = width
#   if height < min_height: min_height = height
#   if width < min_width : min_width = width

# print("mean of height :",heights/len(all_labels))
# print("mean of width :",widths/len(all_labels))

# get details with size DF

In [21]:
height_list = []
width_list = []
for image_path in all_labels['image_path']:
  height = Image.open(image_path).height
  width = Image.open(image_path).width
  
  height_list.append(height)
  width_list.append(width)

sizes_df = pd.DataFrame({'width':width_list,'height':height_list})
sizes_df.describe()

,width,height
count,11719.000000,11719.000000
mean,172.944705,54.925250
std,44.082070,9.066699
min,40.000000,28.000000
25%,144.000000,49.000000
50%,171.000000,54.000000
75%,199.000000,61.000000
max,363.000000,108.000000


# I think no way exist except trying different sizes

so i'm going to define a resize function

In [22]:
import numpy as np

new_dir_path = './images/'
try:
    os.makedirs(new_dir_path + 'test/')
    os.makedirs(new_dir_path + 'train/')
except:print("path already exists")


path already exists


# Early spliting

In [6]:

# def move_to_new_dir(df):#,width,height):
  
#   print("train size : 0 to ",round(len(df)*0.8))
#   for i in range(round(len(df)*0.8)):
#     im = Image.open(df['image_path'][i])
#     # img_resized = im.resize((width,height))
#     im.save(new_dir_path + 'train/'+ "{:0>{}}".format(i, 5) + ".png")
#     #img_resized.save(path.replace(".png","_r.png"))

#   print(f"test size : {round(len(df)*0.8)} to {len(df)}")
#   for i in range(round(len(df)*0.8),len(df)):
#     im = Image.open(df['image_path'][i])
#     # img_resized = im.resize((width,height))
#     im.save(new_dir_path +'test/'+ "{:0>{}}".format(i, 5) + ".png")
#     #img_resized.save(path.replace(".png","_r.png"))


# #resize_images(all_labels,256,256)
# move_to_new_dir(all_labels)#,363,108)


train size : 0 to  9375
test size : 9375 to 11719


In [23]:
# store all images into one directory with renameing and updating dataframe
def move_to_im_dir(df):
  for i,img in enumerate(df['image_path']):
    im = Image.open(img)
    im.save(new_dir_path + "{:0>{}}.png".format(i, 5))

move_to_im_dir(all_labels)
all_labels["new_path"] = ["{}{:0>{}}.png".format(new_dir_path,i, 5) for i in range(len(all_labels))]
all_labels = all_labels.drop(columns=['image_path'])
all_labels.head(15)
all_labels.tail(15)

,label,new_path
11704,20000,./images/11704.png
11705,20000,./images/11705.png
11706,444665,./images/11706.png
11707,29524,./images/11707.png
11708,6792972,./images/11708.png
11709,170000,./images/11709.png
11710,162000,./images/11710.png
11711,340000,./images/11711.png
11712,11900,./images/11712.png
11713,135000,./images/11713.png


In [ ]:
all_labels['label'] = all_labels['label'].astype(int)
all_labels['label'].describe()

SAVE each pixel in a column

In [ ]:
# images_array = []
# for row in all_labels['image_path']:
#     img = Image.open(row).convert('L')
#     images_array.append(np.array(img)/255.0)

# images_array = np.array(images_array)
# images_df = pd.DataFrame(images_array.reshape(images_array.shape[0], -1))

# column_names = ['pixel'+str(i) for i in range(images_array.shape[1]*images_array.shape[2])]
# images_df.columns = column_names

# images_df

In [ ]:
# label_dummies = pd.get_dummies(all_labels['number'])

# all_labels_encoded = pd.concat([all_labels, label_dummies], axis=1)
# all_labels_encoded.drop(['number','image_path'], axis=1, inplace=True)
# all_labels_encoded

# extract multi label without considering order

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# # Define the sequential model
# model = Sequential([

#     Dense(128, activation='relu'),
#     Dense(len(y.columns), activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model on the training set
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

absolutely with these lenth of data we can't use machine learning algorithm directly

so as dear mentor said wen need to split numbers into digits or use RNN
but at first i have an idea :
create a labels like "first_character_is_1" and "third_character_is_3"
i dont know how this affect

# let's try some ideas :
- multi label without order (digit exists or not)(cant cover duplicate digits)
- multi label with order (is there zero in first position or not)(support any thing except alternative size of text)
- segmentation with simple process like hurtizental croping (this bug is not seprated digits)
- segmentation with MNIST prepaired models(this bug is not clear digits)
- ...


# I found Good Idea for labeling :

but maybe it is very complex because of more dimension

also we have few data for this (i think)

In [ ]:
import numpy as np
import pandas as pd

# Define the labels
labels = ["21064", "12345", "98765"]

# Define the vocabulary of possible values
vocab = list("0123456789")

# Convert the labels to sequences of one-hot encoded values
label_seqs = []
for label in labels:
    seq = np.zeros((len(label), len(vocab)))
    for i, char in enumerate(label):
        seq[i, vocab.index(char)] = 1
    label_seqs.append(seq)

# Create a DataFrame with the label sequences
df = pd.DataFrame({
    "label": labels,
    "sequence": label_seqs
})

print(df)

# let's try it on our dataset
- at first we need to know max number lenth (with ploting lenth maybe we remove large outlier numbers)
- use the above code example for creating label
- match the data frame with new labels

In [30]:
all_labels['number'] = all_labels['number'].astype(int)
labels = np.array(all_labels['number'])
print("min is :",labels.min())
print("max is ",labels.max())

min is : 15
max is  67491288


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create an array of integers

fig = plt.figure(figsize=(28, 50))
# Plot the array
plt.scatter(np.arange(len(labels[:6000])),[i if i < 1e5 else  1e5 for i in labels[:6000]])

# Show the plot
plt.show()

i understand that our first directory images (CAR-A) have lower value than (CAR-B)

so i decide to back at first step and explore more on data origin (also i read the dataset document from veena)
and these datasets comes from different places (autritia and auerogue)


In [48]:
print(np.mean(labels),
np.mean(labels[:6000]),
np.mean(labels[6000:]))

226660.01271439542 156490.48 300277.2878125546


In [ ]:
fig = plt.figure(figsize=(20, 20),dpi=300)
plt.hist([i for i in np.sort(labels)],weights=np.arange(len(labels)), bins=200)
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# Define the labels
labels = ["21064", "12345", "98765"]

# Define the vocabulary of possible values
vocab = list("0123456789")

# Convert the labels to sequences of one-hot encoded values
label_seqs = []
for label in labels:
    seq = np.zeros((len(label), len(vocab)))
    for i, char in enumerate(label):
        seq[i, vocab.index(char)] = 1
    label_seqs.append(seq)

# Create a DataFrame with the label sequences
df = pd.DataFrame({
    "label": labels,
    "sequence": label_seqs
})

print(df)

In [ ]:
%pip install tensorflow
import tensorflow

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True
    )

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


In [ ]:
hi